In [2]:
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, r2_score, roc_auc_score, f1_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.tree import DecisionTreeRegressor
# from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
# from sklearn.tree import plot_tree
# from sklearn.pipeline import Pipeline
# from sklearn.preprocessing import OneHotEncoder 
# from sklearn.compose import make_column_transformer, make_column_selector

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split, KFold

# import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")


In [3]:
df = pd.read_csv(r'C:\Users\DAI.STUDENTSDC\Desktop\Machine Learning\Data Sets\Smookers\train.csv', index_col=0)
df.head()


age  height(cm)  weight(kg)  waist(cm)  eyesight(left)  eyesight(right)  \
id                                                                            
0    55         165          60       81.0             0.5              0.6   
1    70         165          65       89.0             0.6              0.7   
2    20         170          75       81.0             0.4              0.5   
3    35         180          95      105.0             1.5              1.2   
4    30         165          60       80.5             1.5              1.0   

    hearing(left)  hearing(right)  systolic  relaxation  ...  HDL  LDL  \
id                                                       ...             
0               1               1       135          87  ...   40   75   
1               2               2       146          83  ...   57  126   
2               1               1       118          75  ...   45   93   
3               1               1       131          88  ...   38  102   
4               1               1       121          76  ...   44   93   

    hemoglobin  Urine protein  serum creatinine  AST  ALT  Gtp  dental caries  \
id                                                                              
0         16.5              1               1.0   22   25   27              0   
1         16.2              1               1.1   27   23   37              1   
2         17.4              1               0.8   27   31   53              0   
3         15.9              1               1.0   20   27   30              1   
4         15.4              1               0.8   19   13   17              0   

    smoking  
id           
0         1  
1         0  
2         1  
3         0  
4         1  

[5 rows x 23 columns]

In [4]:
df.smoking.value_counts()

smoking
0    89603
1    69653
Name: count, dtype: int64

In [5]:
df.isnull().sum()

age                    0
height(cm)             0
weight(kg)             0
waist(cm)              0
eyesight(left)         0
eyesight(right)        0
hearing(left)          0
hearing(right)         0
systolic               0
relaxation             0
fasting blood sugar    0
Cholesterol            0
triglyceride           0
HDL                    0
LDL                    0
hemoglobin             0
Urine protein          0
serum creatinine       0
AST                    0
ALT                    0
Gtp                    0
dental caries          0
smoking                0
dtype: int64

In [6]:
X = df.drop(columns=['smoking'], axis=1)
y = df['smoking']


In [7]:
# mmsc = MinMaxScaler()
# stsc = StandardScaler()

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=24)

In [ ]:
#With Logistic Regression
log_r = LogisticRegression(random_state=24)
bagg_r = BaggingClassifier(estimator=log_r, random_state=24)

# params = {'BG__n_estimators': [10, 50, 50]}
params = {'estimator__fit_intercept': [True, False],          
          'n_estimators': [10, 50, 100]}

gcv_log_r = GridSearchCV(bagg_r, param_grid=params, cv=kfold, scoring='neg_log_loss', verbose=1)

In [19]:
gcv_log_r.fit(X, y)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('BG',
                                        BaggingClassifier(estimator=LogisticRegression(random_state=24),
                                                          random_state=24))]),
             param_grid={'BG__estimator__fit_intercept': [True, False],
                         'BG__n_estimators': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])},
             scoring='neg_log_loss', verbose=1)

In [20]:
print(gcv_log_r.best_score_)
print(gcv_log_r.best_params_)

-0.5210148433658817
{'BG__estimator__fit_intercept': True, 'BG__n_estimators': 1}


#### With Decision Tree Classifier

In [ ]:

dtr = DecisionTreeClassifier(random_state=24)

bagg_dtr =BaggingClassifier(estimator=dtr, random_state=24)

params = {'estimator__max_depth': [None, 3, 5],
          'estimator__min_samples_split': [2, 5, 10],
          'estimator__min_samples_leaf': [1, 5, 10],
          'n_estimators': [10, 50, 100]}

gcv_dtr = GridSearchCV(bagg_dtr, param_grid=params, cv=kfold, scoring='neg_log_loss', verbose=1)

In [27]:
gcv_dtr.fit(X, y)

Fitting 5 folds for each of 135 candidates, totalling 675 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('BG',
                                        BaggingClassifier(estimator=DecisionTreeClassifier(random_state=24),
                                                          random_state=24))]),
             param_grid={'BG__estimator__max_depth': [None, 3, 5],
                         'BG__estimator__min_samples_leaf': [1, 5, 10],
                         'BG__estimator__min_samples_split': [2, 5, 10],
                         'BG__n_estimators': array([ 1,  3,  5,  7, 10])},
             scoring='neg_log_loss', verbose=1)

In [28]:
print(gcv_dtr.best_score_)
print(gcv_dtr.best_params_)

-0.4836880050225435
{'BG__estimator__max_depth': 5, 'BG__estimator__min_samples_leaf': 5, 'BG__estimator__min_samples_split': 2, 'BG__n_estimators': 10}


#### With SVC

In [ ]:
svm = SVC(random_state=24)
bagg_svm = BaggingClassifier(estimator=svm, random_state=24)
# bagg_svm.get_params()

params = { 'estimator__C': np.linspace(0, 1, 5),
          'estimator__decision_function_shape': ['ovr', 'ovo'],
          'n_estimators': [10, 50, 100]}

gcv_svm = GridSearchCV(bagg_svm, param_grid=params, cv=kfold,scoring='neg_log_loss', verbose=1)


In [ ]:
gcv_svm.fit(X, y)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


In [1]:
print(gcv_svm.best_score_)
print(gcv_svm.best_params_)

NameError: name 'gcv_svm' is not defined